# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data = pd.read_csv("../output_data/cities.csv")
data.head()


,Unnamed: 0,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,0,Tiznit Province,29.58,-9.50,63.73,84,67,0.87,MA,1603258404
1,1,Guerrero Negro,27.98,-114.06,68.34,90,0,9.24,MX,1603258404
2,2,Dikson,73.51,80.55,24.91,97,95,5.41,RU,1603258405
3,3,Yellowknife,62.46,-114.35,23.00,73,90,9.17,CA,1603258290
4,4,Ushuaia,-54.80,-68.30,39.20,86,40,8.34,AR,1603258405


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
loc_data = data[["Lat", "Lng"]].astype(float)
hum_data = data["Humidity"].astype(float)


In [4]:
m = gmaps.Map()

In [5]:
heat_layer = gmaps.heatmap_layer(loc_data, weights = hum_data, dissipating = False, max_intensity = 100, point_radius = 2.5)
m.add_layer(heat_layer)
m

Map(configuration={'api_key': None}, data_bounds=[(-45.61929368532762, -179.99999), (85.0, 179.99999)], layers…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
data1 = data[data["Max_Temp"] <= 80]
data2 = data1[data1["Max_Temp"]>= 70]
data3 = data2[data2["Wind_Speed"] <= 10]
data_final = data3[data3["Cloudiness"] == 0]
data_final

,Unnamed: 0,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
7,7,Adrar,20.50,-10.07,79.38,20,0,5.06,MR,1603258406
127,127,Damietta,31.42,31.81,79.00,63,0,5.61,EG,1603258430
146,146,Najrān,17.49,44.13,75.20,17,0,5.23,SA,1603258434
161,161,Broken Hill,-31.95,141.43,78.80,16,0,9.17,AU,1603258437
179,179,Álamos,27.02,-108.93,73.00,80,0,1.01,MX,1603258442
183,183,Riyadh,24.69,46.72,71.60,26,0,2.24,SA,1603258393
202,202,Taoudenni,22.68,-3.98,74.44,20,0,8.30,ML,1603258446
205,205,Jalingo,8.88,11.37,75.54,84,0,2.75,NG,1603258447
219,219,Ténenkou,14.46,-4.92,78.58,26,0,0.92,ML,1603258450
244,244,Kandi,11.13,2.94,73.60,76,0,4.03,BJ,1603258456


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = data_final[["City","Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df.head()

C:\Users\staff\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,Lat,Lng,Hotel Name
7,Adrar,MR,20.50,-10.07,
127,Damietta,EG,31.42,31.81,
146,Najrān,SA,17.49,44.13,
161,Broken Hill,AU,-31.95,141.43,
179,Álamos,MX,27.02,-108.93,


In [8]:
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][1]["name"]
    except:
        hotel_df.loc[index, "Hotel Name"] = 'N/A'
        
hotel_df

C:\Users\staff\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,City,Country,Lat,Lng,Hotel Name
7,Adrar,MR,20.50,-10.07,N/A
127,Damietta,EG,31.42,31.81,برجولات خشبية - Wooden Pergolas
146,Najrān,SA,17.49,44.13,Ersal Money Transfer
161,Broken Hill,AU,-31.95,141.43,Daydream Motel & Cottages
179,Álamos,MX,27.02,-108.93,Luz del Sol
183,Riyadh,SA,24.69,46.72,InterContinental Riyadh
202,Taoudenni,ML,22.68,-3.98,Garage Toumboctou Taoudani
205,Jalingo,NG,8.88,11.37,Polaris Bank Limited
219,Ténenkou,ML,14.46,-4.92,Hopital
244,Kandi,BJ,11.13,2.94,APIDA KANDI


In [9]:
hotel_df = hotel_df[hotel_df["Hotel Name"] != "N/A"]
hotel_df

,City,Country,Lat,Lng,Hotel Name
127,Damietta,EG,31.42,31.81,برجولات خشبية - Wooden Pergolas
146,Najrān,SA,17.49,44.13,Ersal Money Transfer
161,Broken Hill,AU,-31.95,141.43,Daydream Motel & Cottages
179,Álamos,MX,27.02,-108.93,Luz del Sol
183,Riyadh,SA,24.69,46.72,InterContinental Riyadh
202,Taoudenni,ML,22.68,-3.98,Garage Toumboctou Taoudani
205,Jalingo,NG,8.88,11.37,Polaris Bank Limited
219,Ténenkou,ML,14.46,-4.92,Hopital
244,Kandi,BJ,11.13,2.94,APIDA KANDI
250,Wana,PK,32.30,69.57,UAE PAP Cadet College Wana E2-2


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content = hotel_info )
m.add_layer(hotel_layer)

# Display figure

m

Map(configuration={'api_key': None}, data_bounds=[(-45.61929368532762, -179.99999), (85.0, 179.99999)], layers…